In [1]:
import aiohttp
import asyncio
import bs4
import datetime
from io import BytesIO
import pandas as pd
import PyPDF2
import requests
from tqdm.contrib.telegram import tqdm

from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar, LTTextLineHorizontal
from PyPDF2 import PdfReader

from urllib.parse import unquote
from transliterate import to_latin
import os


In [2]:
TOKEN = os.getenv('TOKEN')
CHAT_ID = os.getenv('CHAT_ID')

In [35]:
tqdm.pandas()

In [17]:

# scraper.py
from urllib.parse import unquote
import requests
from bs4 import BeautifulSoup

url = 'https://e-library.namdu.uz/1.%20%D0%8E%D0%B7%D0%B1%D0%B5%D0%BA%D0%B8%D1%81%D1%82%D0%BE%D0%BD%20%D0%A0%D0%B5%D1%81%D0%BF%D1%83%D0%B1%D0%BB%D0%B8%D0%BA%D0%B0%D1%81%D0%B8%20%D0%B1%D0%B8%D1%80%D0%B8%D0%BD%D1%87%D0%B8%20%D0%9F%D1%80%D0%B5%D0%B7%D0%B8%D0%B4%D0%B5%D0%BD%D1%82%D0%B8%20%D0%98.%D0%90.%20%D0%9A%D0%B0%D1%80%D0%B8%D0%BC%D0%BE%D0%B2%20%D0%B0%D1%81%D0%B0%D1%80%D0%BB%D0%B0%D1%80%D0%B8/'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')
quotes = soup.find('table', {"id": "indexlist"}).find_all('tr')

for i in quotes[1:]:
    link = unquote(i.find("a")['href'])
    if link[-3:] == "pdf":
        
        print("PDF", link)
    elif link[-1:] == '/':
        print("folder", link)
# print(quotes)
   

folder /
folder ЮКСАК МАЪНАВИЯТ ЕНГИЛМАС КУЧ/
folder o`zbekiston 21 asr busag`asida/
PDF Ўзбекистон Мустақилликка эришиш остонасида И.Каримов.pdf
PDF Ўзбекистон мустақилликка эришиш остонасида. Каримов И.pdf
PDF Каримов И.А. Замонавий уй-жой қурулиши-қишлоқ жойларни комплекс ривожлантириш., pdf.pdf
PDF Наша высщая цуль - независомость и процветание родины, свобода и блогополучие народа. Каримов И.pdf
PDF Янгича фикрлаш ва ишлаш-давр талаби. И.Каримов.pdf
PDF 1-т Узбекистон миллий иктиклол сиёсат мафкура.pdf
PDF 1 tom Karimov I. O`zbekiston.Milliy istiqlol iqtisod siyosat mafkura. kitob. T. 1993 .pdf.pdf
PDF 2-т Биздан озод ва обод ватан колсин.pdf
PDF 3-т Ватан саждагох каби мукаддасдир.pdf
PDF 4-т Бунёдкорлик йўлидан.pdf
PDF 6-т Хавфсизлик ва баркарор тараккиёт йулида.pdf
PDF 7-т Биз келажагимизни уз кулимиз билан курамиз.pdf
PDF 8-т Озод ва обод ватан, эркин ва фаровон хаёт-максдадимиз.pdf
PDF 9-т Ватан равнаки учун хар биримиз масъулмиз.pdf
PDF 10-т Хавфсизлик ва тинчлик учун курашм

In [97]:
visited_urls = set()

def get_pdf_links(url):
    pdf_links = []
    
    try:
        if url in visited_urls:
            return pdf_links

        visited_urls.add(url)

        page = requests.get(url)
        if page.status_code == 200:
            text = page.text
            soup = bs4.BeautifulSoup(text, "lxml")
            quotes = soup.find('table', {"id": "indexlist"}).find_all('tr')

            leng = len(quotes)

            for i in tqdm(range(1, leng), token=TOKEN, chat_id=CHAT_ID):
                link = unquote(quotes[i].find("a"))
                if link['href'].endswith('.pdf'):
                    file_url = urljoin(url, link['href'])
                    pdf_links.append(file_url)
                elif link['href'].endswith('/'):
                    folder_url = urljoin(url, link['href'])
                    pdf_links.extend(get_pdf_links(folder_url))
        return pdf_links
    
    except Exception as e:
            print(e)

In [98]:
base_url = "https://e-library.namdu.uz/"


In [102]:
with open('HelperFiles/namdu.txt', 'w') as file:
    file.write('\n'.join(all_pdf_links))

In [129]:
with open('HelperFiles/namdu_uz.txt', 'r') as file:
    namdu_books = [book.replace("\n", "") for book in file.readlines()]

# Download pdf files

In [134]:
def main():
    for link in tqdm(namdu_books[1:], token=TOKEN, chat_id=CHAT_ID):
        try:
            resp = requests.get(link)
            if resp.status_code == 200:
                text = resp.content
                with open('Namdu_books/'+unquote(link.split('/')[-1]), 'wb') as file:
                    file.write(text)
#             elif resp.status != 404:
#                 await asyncio.sleep(60) # 1 minute
        except Exception as e:
            print(e, link)

In [153]:
folder = "./Namdu_books/"

In [2]:
# for file in os.listdir(folder):
#     try:
#         filename = file.rsplit(".", 1)[0]
#         for i in [f"({num})" for num in range(1,10)]:
#             if i in filename:
#                 print(filename)
#                 os.remove(folder+file)
# #         if file.rsplit(".", 1)[0] not in elib_uzb:
# #             
# #             print(folder + file)
#     except Exception as e:
#         print(e)

# READ THE BOOKS

In [13]:
import io
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
import glob

In [14]:
def extract_text_from_pdf(file_path):
#     try:
    resource_manager = PDFResourceManager()
    output_string = io.StringIO()
    codec = 'utf-8'
    laparams = None
    device = TextConverter(resource_manager, output_string, codec=codec, laparams=laparams)
    with open(file_path, 'rb') as file:
        interpreter = PDFPageInterpreter(resource_manager, device)
        for page in PDFPage.get_pages(file, check_extractable=True):
            interpreter.process_page(page)
        text = output_string.getvalue()
    device.close()
    output_string.close()
    return text
#     except

replaces = {"ª" : "O'", "¡" : "G'", "²" : "H", "¯" : "Q", "^" : "q", "\xad" : "", "\'":"'"}
def replace_values(text):
    for i,j in replaces.items():
        text = text.replace(i, j)
    return text

In [159]:
pdfs = glob.glob("Namdu_books/*.pdf")

In [163]:
df1 = pd.DataFrame(results)

df1.to_pickle("NamDU.pickle")

In [33]:
from gradio_client import Client


def uz_detect(text):
    try:
        client = Client("https://azzizz17-api-2.hf.space/")
        result = client.predict(
            text,  # str in 'text' Textbox component
            api_name="/predict"
            )
        result = eval(result)

        if result[0]['label'] in ['uz', 'uz_kr']:
            return text
        return ""
    
    except Exception as e:
        return ""

In [11]:
def replace_values(text):
    replaces = {"ʹ" : "'", "ʺ" : '"', "ʻ" : "'", "ʼ" : "'", "ʽ" : "'", 'ˮ' : '"',  "`":"'", "‘":"'", 
                "’":"'", "‚":",", "‛":"'", '“':'"', '”':'"', '“':'"', '„':',', '‟': '"', "․": ".", "‥":'.', "…":".", "′":"'",
               "″":'"', '‴':'"'}
    
    for i,j in replaces.items():
        text = text.replace(i, j)
    return text

In [28]:
def filter_text(text):
    texts = text.split(".")
    
    for i, t in enumerate(texts):
        for char in t:
            if ord(char)> 122:
                texts[i] = ""
                break
        
    
    texts = [text for text in texts if text != ""]
    
    return ".".join(texts)

In [45]:
def slice_text(text):
    texts = text.split(" ")[50:80]
    
    return " ".join(texts)
    

In [49]:
df = pd.read_pickle("./MainFiles/NamDU_books.pickle")

In [50]:
df['content'] = df['content'].progress_apply(lambda x: replace_values(x))

  0%|          | 0/7206 [00:00<?, ?it/s]

In [51]:
df

,title,content
0,Ekologik_turizm_Pardayev_A_X_Oʻ_q,1 O'ZBEKISTON RESPUBLIKASI OLIY VA O'RTA MAXSU...
1,Shuxrat Eshmurodov. Oʻzlikni anglash va muvaff...,O'zlikni anglash va muvaffaiyatga erishish ilm...
2,4-741,"K,OC IMOV A O., Ye'SIMOV T.9 / ~ *■* *I vd-L. ..."
3,Ekskursiya maʼruza,1 11. Ma`ruza matni 1-mavzu: Ekskursiya va sa...
4,olij_algebra_kursining_metodik_taminoti_uchun_...,44 6) ibitnRi)32(1-raund o'ng qismi. 7) )3...
...,...,...
8535,Iqtisodiy_talimotlar_tarixi_Xajiye_[@iqtisodch...,O'ZBEKISTON RESPUBLIKASI OLIY VA O'RTA MAXSUS ...
8536,Metrologiya standartlashtirish va sertifikatla...,O'ZBEKISTON RESPUBLlKASI OLIY VA O'RTA MAXSUS ...
8538,67. Odam savdosi jinoyatning jinoyat huquqiy v...,Y.RAXIMOV F.SOBIROVODAM SAVDOSI JINOYaTINING J...
8539,65. Xufyona iqtisodiyot. Ortiqov A,UZBEK ISTON RESPUBLIKASI IChKI IShLAR VAZIRLIG...


In [52]:
filter_text(df.iloc[0].content)

'1 O\'ZBEKISTON RESPUBLIKASI OLIY VA O\'RTA MAXSUS TA\'LIM VAZIRLIGI    TOSHKENT DAVLAT IQTISODIYOT UNIVERSITETI      PARDAYeV A.X.,  NORCHAYeV A.N.,  RABBIMOV E.T.      E K O L O G I K   T U R I Z M   O\'quv qo\'llanma           Toshkent-2011    2   Mas\'ul muharrir: i.f.n., dos. A.Eshtayev Taqrizchilar: i.f.d, prof. R.X. Shodiev, Abdurahmonov Sh.A.    Pardayev A.X., Norchayev A.N., Rabbimov E.N.  Ekologik turizm. -O\'quv qo\'llanma.: TDIU, 2011.  -128 b.             O\'quv qo\'llanma O\'zbekiston Respublikasi mustaqilligining 20 yilligiga va Toshkent Davlat iqtisodiyot universitetining 80 yillik yubileyiga bag\'ishlangan bo\'lib, respublikamizdagi turizm va xizmat ko\'rsatish yo\'nalishida mutaxassislar tayyorlaydigan Oliy o\'quv yurtlari o\'quv dasturiga kiritilgan. Ekologik turizm yangi yo\'nalishdagi fanlar qatoriga kirib, turizm va xizmat ko\'rsatish yo\'nalishida tahsil olayotgan talabalarga, mutaxassislarga ekologik turizm faoliyatini o\'rganish, tahlil qilish va ularning faoli

In [53]:
df['content'] = df['content'].progress_apply(lambda x: filter_text(x))

  0%|          | 0/7206 [00:00<?, ?it/s]

In [54]:
df

,title,content
0,Ekologik_turizm_Pardayev_A_X_Oʻ_q,1 O'ZBEKISTON RESPUBLIKASI OLIY VA O'RTA MAXSU...
1,Shuxrat Eshmurodov. Oʻzlikni anglash va muvaff...,O'zlikni anglash va muvaffaiyatga erishish ilm...
2,4-741,"K,OC IMOV A O., Ye'SIMOV T. 1o*- : 'KU mumiy k..."
3,Ekskursiya maʼruza,1 11. 2. Ekskursiya va sayohat atamalarining i...
4,olij_algebra_kursining_metodik_taminoti_uchun_...,Bu esa n ga bog'liq holda bir necha Feystel t...
...,...,...
8535,Iqtisodiy_talimotlar_tarixi_Xajiye_[@iqtisodch...,O'ZBEKISTON RESPUBLIKASI OLIY VA O'RTA MAXSUS ...
8536,Metrologiya standartlashtirish va sertifikatla...,O'ZBEKISTON RESPUBLlKASI OLIY VA O'RTA MAXSUS ...
8538,67. Odam savdosi jinoyatning jinoyat huquqiy v...,"Y.RAXIMOV F. RAXIMOV F.)Y.Raximov, F.SobirovOd..."
8539,65. Xufyona iqtisodiyot. Ortiqov A,UZBEK ISTON RESPUBLIKASI IChKI IShLAR VAZIRLIG...


In [55]:
df2 = df.copy()

In [56]:
df2['content'] = df2['content'].progress_apply(lambda x: slice_text(x))

  0%|          | 0/7206 [00:00<?, ?it/s]

In [62]:
df2.replace(r'^\s*$', float('nan'), regex=True, inplace=True)

df2.dropna(axis=0, how='any', inplace=True)

In [63]:
df2.shape

(6833, 2)

In [65]:
df[df['title'].isin(df2['title'])].to_pickle("Namdu_smaller.pickle")

In [59]:
df2['content'] = df2['content'].progress_apply(lambda x: uz_detect(x))

  0%|          | 0/7206 [00:00<?, ?it/s]

Loaded as API: https://azzizz17-api-2.hf.space/ ✔
Loaded as API: https://azzizz17-api-2.hf.space/ ✔
Loaded as API: https://azzizz17-api-2.hf.space/ ✔
Loaded as API: https://azzizz17-api-2.hf.space/ ✔
Loaded as API: https://azzizz17-api-2.hf.space/ ✔
Loaded as API: https://azzizz17-api-2.hf.space/ ✔
Loaded as API: https://azzizz17-api-2.hf.space/ ✔
Loaded as API: https://azzizz17-api-2.hf.space/ ✔
Loaded as API: https://azzizz17-api-2.hf.space/ ✔
Loaded as API: https://azzizz17-api-2.hf.space/ ✔
Loaded as API: https://azzizz17-api-2.hf.space/ ✔
Loaded as API: https://azzizz17-api-2.hf.space/ ✔
Loaded as API: https://azzizz17-api-2.hf.space/ ✔
Loaded as API: https://azzizz17-api-2.hf.space/ ✔
Loaded as API: https://azzizz17-api-2.hf.space/ ✔
Loaded as API: https://azzizz17-api-2.hf.space/ ✔
Loaded as API: https://azzizz17-api-2.hf.space/ ✔
Loaded as API: https://azzizz17-api-2.hf.space/ ✔
Loaded as API: https://azzizz17-api-2.hf.space/ ✔
Loaded as API: https://azzizz17-api-2.hf.space/ ✔


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Loaded as API: https://azzizz17-api-2.hf.space/ ✔
Loaded as API: https://azzizz17-api-2.hf.space/ ✔
Loaded as API: https://azzizz17-api-2.hf.space/ ✔
Loaded as API: https://azzizz17-api-2.hf.space/ ✔
Loaded as API: https://azzizz17-api-2.hf.space/ ✔
Loaded as API: https://azzizz17-api-2.hf.space/ ✔
Loaded as API: https://azzizz17-api-2.hf.space/ ✔
Loaded as API: https://azzizz17-api-2.hf.space/ ✔
Loaded as API: https://azzizz17-api-2.hf.space/ ✔
Loaded as API: https://azzizz17-api-2.hf.space/ ✔
Loaded as API: https://azzizz17-api-2.hf.space/ ✔
Loaded as API: https://azzizz17-api-2.hf.space/ ✔
Loaded as API: https://azzizz17-api-2.hf.space/ ✔
Loaded as API: https://azzizz17-api-2.hf.space/ ✔
Loaded as API: https://azzizz17-api-2.hf.space/ ✔
Loaded as API: https://azzizz17-api-2.hf.space/ ✔
Loaded as API: https://azzizz17-api-2.hf.space/ ✔
Loaded as API: https://azzizz17-api-2.hf.space/ ✔
Loaded as API: https://azzizz17-api-2.hf.space/ ✔
Loaded as API: https://azzizz17-api-2.hf.space/ ✔


In [61]:
df2.to_pickle("Namdu_smaller.pickle")

In [67]:
df = pd.read_pickle("MainFiles/Namdu_smaller.pickle")

In [72]:
df.iloc[5]['content']

' Eron, Irok, OzarbayjonSho>fux Mirzo (1377-1447) Xuroson, Mozandaron, Sijiston (1405-1409); Xuroson, Fap6.RIRIYaTI TOShKENT - 2005Forscha matndan Alixontura Soyeuniy va Xdbibulla Karomatov tarjimasi. Izox,lar Buriboy Axmedov tomonidan tayyorlangan.Temur tuzuklari /Suzboshi muallifi va mas\'ul muxarrir M. Ali, forchadan tarj. A.SoBuniy va X.63.Buyuk obidaga edtirom"Temur tuzuklari" kitobining yozilganiga olti yuz yil tuldiIAmir Temur Kuragon tarix maydonida javlon urgan jaxonshumul siymolardan biridir. Biz uning tarixiga, favkulodsa shaxsiga, avlodlari faoliyatiga kayta-kayta murojaat kilaveramiz. Tarixga yuz burib, kaysi yullar- dan yurmaylik, kachon bulmasin, kayerda bulmasin, yuli- miz aylanib baribir ushbu mu\'tabar zot x;uzuriga olib kelaveradi. Ushbu ulur xizmatlari uchun xalkimiz undan bexad minnatdordir. Vaktlar keladiki, avlodlar xali soxibkiron Dmir Temur xayotining xar bir kunini, xar bir laxzasini bitta xam koldirmay chukur urganib chikadilar. Bu borada, ayniksa, bevosita u

In [71]:
df['title'].head(20)

0                     Ekologik_turizm_Pardayev_A_X_Oʻ_q
1     Shuxrat Eshmurodov. Oʻzlikni anglash va muvaff...
2                                                 4-741
3                                    Ekskursiya maʼruza
4     olij_algebra_kursining_metodik_taminoti_uchun_...
5     Temur tuzuklari. Xalqaro Amir Temur jamgʻarmas...
6                    Maxsus psixologiya. Mo'minova L.R.
7                            63. Tarixiy manbashunoslik
9     22. Matematik mantiq va diskret matematika 2-j...
10    Struktur tilshunoslik. Ildizlari va yo'nalishl...
11    Q.Usmonov, M.Sodiqov, S.Burxonova. O'zbekiston...
12    65. Oʻzbekistonning iqtisodiy va ijtimoiy joʻg...
13                    Qurbanov.X. Korporativnыe finansы
14    Davletov_I_X_Qurilishda_marketing_O'quv_qo'lla...
15                          Madraimov A. Manbashunoslik
18              saida_zunnunova_tanlangan_asarlar_1_tom
19                 67. Ekologiya huquqi. Chuboyeva O.K.
20                              Xorijiy investit

In [84]:
df5 = pd.read_pickle("MainFiles/DTM.pickle")

In [85]:
df5

0                                                        
1       ️Buni hamma eshitishi kerak ekan...  — Diniy m...
4       00:00 ●━━───── 05:09    ⇆ㅤㅤ◁ㅤ❚❚ㅤ▷ㅤㅤ↻  Avaz Oxu...
5                                                        
6                                     — Diniy maxsulotlar
                              ...                        
2524    ️ 2023-yil kunlari barcha hududlarda 8 ta OTMn...
2525     Hoy abituriyent uxlama,Yengilmagin g'aflatga....
2526       Bugun yakshanba,  kuni platformasida  navba...
2527    ️ .Bir oliy ta’lim muassasasi doirasida oliy t...
2528       birinchi smena soat 9:00 da;  ikkinchi smen...
Name: Text, Length: 1249, dtype: object

In [82]:
df4 = df4.apply(lambda x: filter_text(x))

In [83]:
df4

0                                                        
1                                                        
4                                                        
5                                                        
6                                                        
                              ...                        
2524     Abituriyentlar qachon, qayerda imtihonda isht...
2525     Hoy abituriyent uxlama,Yengilmagin g'aflatga....
2526       Bugun yakshanba,  kuni platformasida  navba...
2527                                                     
2528       birinchi smena soat 9:00 da;  ikkinchi smen...
Name: Text, Length: 1249, dtype: object

In [77]:
df5 = pd.read_pickle("MainFiles/EDU.pickle")

In [79]:
df5[50]

': 2+2=5'